# Challenge: Data cleaning & validation

Unit 1: Lesson 3: Project 6

Determine the five most common journals and the total articles for each.

Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

For a real bonus round, identify the open access prices paid by subject area.

In [1]:
import numpy as np
import pandas as pd
import re

In [55]:
#Changing the encoding - but no idea what happened here (did I just nuke all the bad data?)
trust = pd.read_csv("APC.csv", error_bad_lines=False, encoding = "ISO-8859-1")
trust.head(100)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [14]:
trust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [16]:
trust['Journal title'].describe()

count         2126
unique         984
top       PLoS One
freq            92
Name: Journal title, dtype: object

In [17]:
# Split by ID, but this identifies the article not the Journal
IDsplit = trust['PMID/PMCID'].str.split(' ', expand=False)

In [56]:
# Clean Journal titles
trust['Journal'] = trust['Journal title'].str.strip()

#Remove 'J', 'J of' and 'Journal of'
trust['Journal'] = trust['Journal'].str.replace('Journal of ', '')
trust['Journal'] = trust['Journal'].str.replace('J of ', '')
trust['Journal'] = trust['Journal'].str.replace('J ', '')

#all lowercase
trust['Journal'] = trust['Journal'].str.lower()


# some other spelling and spacing issues
trust['Journal'] = trust['Journal'].str.replace('plosone', 'plos one')
trust['Journal'] = trust['Journal'].str.replace('medical chemistry', 'med chem')
trust['Journal'] = trust['Journal'].str.replace('organic chemistry', 'org chem')
trust['Journal'] = trust['Journal'].str.replace('antimicrobial agfents chemotherapy', 'antimicrobial agents chemotherapy')
trust['Journal'] = trust['Journal'].str.replace('journal od clinical endocrinology', 'clinical endocrinology')
trust['Journal'] = trust['Journal'].str.replace('Antimicrobial Agfents and Chemotherapy', 'Antimicrobial Agents and Chemotherapy')


# remove 'and' and '&'
trust['Journal'] = trust['Journal'].str.replace('and ', '')
trust['Journal'] = trust['Journal'].str.replace('& ', '')

# remove 'the' and '.'
trust['Journal'] = trust['Journal'].str.replace('the ', '')
trust['Journal'] = trust['Journal'].str.replace('.', '')



trust['Journal'].describe()

count         2126
unique         841
top       plos one
freq           199
Name: Journal, dtype: object

In [59]:
# Still 841 unique entries and lots of cleaning, but maybe possible to already tell top 5 journals
quickcount = trust.groupby('Journal').count()
quickcount.head(100)

,PMID/PMCID,Publisher,Journal title,Article title,Cost
Journal,,,,,
abnormal child psychology psychiatry,1,1,1,1,1
abnormal psychology,1,1,1,1,1
academy of nutrition dietetics,1,1,1,1,1
acquired immune deficiency syndromes,1,1,1,1,1
acquired immune deficiency syndroms (jaids),1,1,1,1,1
acs chemical biology,4,5,5,5,5
acs chemical neuroscience,1,1,1,1,1
acs nano,2,2,2,2,2
"acta crystallographica section d, biological crystallography",1,1,1,1,1


In [58]:
# Sorting, not clean yet, but a good idea of top 5 journals
quickcount.sort_values('Article title', ascending=False)

,PMID/PMCID,Publisher,Journal title,Article title,Cost
Journal,,,,,
plos one,197,199,199,199,191
biological chemistry,56,57,57,57,56
neuroimage,28,29,29,29,29
nucleic acids research,25,26,26,26,26
plos genetics,23,24,24,24,22
plos pathogens,24,24,24,24,16
proceedings of national academy of sciences,22,22,22,22,22
plos neglected tropical diseases,20,20,20,20,20
human molecular genetics,17,19,19,19,18


In [57]:
#Rename column heading
trust = trust.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'})

#Take away sterling sign
trust['Cost'] = trust['Cost'].str.replace('£', '')

#Convert to numeric
trust['Cost'] = pd.to_numeric(trust['Cost'], errors='coerce')

trust['Cost'].describe()

count      2114.000000
mean      24206.441892
std      147300.992149
min           0.000000
25%        1280.000000
50%        1889.950000
75%        2322.315000
max      999999.000000
Name: Cost, dtype: float64

In [21]:
print(trust['Cost'].max())
print(trust['Cost'].min())

999999.0
0.0


In [25]:
trust.loc[trust['Cost']==999999.0, 'Cost'] = np.NaN

In [50]:
# Mean
quickCost = trust.groupby('Journal').mean()
# Journal column becomes index! So sorted alphabetically anyway
# All Journals groupby will be the same, so expand df to include median and std
#quickCost.sort_values('Journal', ascending=True, inplace=True)
quickCost['Median'] = trust.groupby('Journal').median()['Cost']
quickCost['Std']= trust.groupby('Journal').std()['Cost']


#Median
#quickMedian = trust.groupby('Journal').median()


#Std
#quickStd = trust.groupby('Journal').std()



In [53]:
quickCost.sort_values('Cost', ascending=False, inplace=True)
quickCost.head(20)

,Cost,Median,Std
Journal,,,
movement disorders,15176.788000,2010.240,51414.460186
public service review,6000.000000,6000.000,NaN
lancet neurology,5040.000000,5040.000,1018.233765
curr biol,4151.770000,4151.770,NaN
cell journal,4041.050000,4041.050,NaN
cell host microbe,4032.460000,4032.460,273.763461
immunity,3934.750000,3934.750,190.791552
molecular cell,3925.736667,3906.430,213.655238
cell metabolism,3924.260000,3904.460,151.324659


In [66]:
# Merge quickcount and quickCost to have everything together
all = quickcount.merge(quickCost, left_index=True, right_index=True, how='left')
all = all.rename(columns={'Cost_x':'Count of Cost', 'Cost_y':'Average Cost'})
all.sort_values('Journal title', inplace=True, ascending=False) 
all.head()

,PMID/PMCID,Publisher,Journal title,Article title,Count of Cost,Average Cost,Median,Std
Journal,,,,,,,,
plos one,197,199,199,199,199,1935.641204,894.71,13873.278246
biological chemistry,56,57,57,57,57,1406.703214,1299.74,403.255865
neuroimage,28,29,29,29,29,2215.168276,2326.43,266.653947
nucleic acids research,25,26,26,26,26,1149.000000,852.00,442.940447
plos genetics,23,24,24,24,24,1643.110909,1712.73,153.366825
